In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.0f}'.format

In [2]:
# 분류 테이블
claf_pnc = pd.read_excel('C:/FSS_FinesReport/Setting/RBC분류.xlsx', sheet_name='일반')
claf_auto = pd.read_excel('C:/FSS_FinesReport/Setting/RBC분류.xlsx', sheet_name='자동차')

In [3]:
def genDates(date):
    # 필요한 업무보고서 일자 생성
    date = str(date)
    if date[2:] == '12':
        res = [date, str(int(date) - 100), str(int(date) - 200)]
    else:
        res = [date, str(int(date) - 100)[:2] + '12', str(int(date) - 100), str(int(date) - 200)[:2] + '12', str(int(date) - 200), str(int(date) - 300)[:2] + '12', str(int(date) - 300)]
    return res

In [22]:
def fss_rpt(num, date):
    # 업무보고서와 기준일자를 받아서 테이블을 출력하는 함수
    num = str(num)
    while len(num) < 3: num = '0' + num
    rpt = pd.read_excel('C:/FSS_FinesReport/Setting/업무보고서_'+str(date)+'.xlsx', sheet_name='AI'+num)
    
    if num == '059':
        rpt = rpt.iloc[11:,1:]
        rpt.index = pd.RangeIndex(start=14,stop=155, step=1)
        rpt.columns = pd.RangeIndex(start=2,stop=80, step=1)
        rpt.iloc[:,1:] = rpt.iloc[:,1:].astype(float, inplace=True).fillna(0)
    elif num == '060':
        rpt = pd.read_excel('C:/FSS_FinesReport/Setting/업무보고서_'+str(1712)+'.xlsx', sheet_name='AI060')
        rpt = rpt.iloc[11:]
        rpt.index = pd.RangeIndex(start=14,stop=146, step=1)
        rpt.columns = pd.RangeIndex(start=1,stop=60, step=1)
        rpt.iloc[:,1:] = rpt.iloc[:,1:].astype(float, inplace=True).fillna(0)
    else:
        print('AI{:}_{:}는 목록에 없습니다'.format(num,date))
        return None
    
    print('[AI{:}_{:}]가 생성 되었습니다.'.format(num,date))
    return rpt

In [34]:
def fss_filter(num, date):
    # 사업실적표를 받아서 필요한 열만 추출하는 함수
    rpt = fss_rpt(num, date)
    num = str(num)
    while len(num) < 3: num = '0' + num
    if num == '059':
        cols = ['RBC분류','모니터링분류','구분',7,8,12,13,17,18,23,32,45,52,55,67]
        colnames = ['RBC분류','모니터링분류','구분','원수보험료','원수해약환급금','수재보험료','수재해약환급금','출재보험료','출재해약환급금','경과보험료','원수지급준비금','수재지급준비금','출재지급준비금','발생손해액','순사업비']
        filtered = rpt.merge(claf_pnc, left_on=2, right_on='구분', how='inner').drop(2, axis=1)[cols]
        filtered.columns = colnames
        filtered = filtered.dropna()
        filtered['원수보험료'] = filtered['원수보험료'] - filtered['원수해약환급금']
        filtered['수재보험료'] = filtered['수재보험료'] - filtered['수재해약환급금']
        filtered['출재보험료'] = filtered['출재보험료'] - filtered['출재해약환급금']
        filtered.drop(['원수해약환급금','수재해약환급금','출재해약환급금'], axis=1, inplace=True)
    if num == '060':
        cols = ['RBC분류','모니터링분류','구분',5,6,10,11,15,16,21,27,34,41,44,58]
        colnames = ['RBC분류','모니터링분류','구분','원수보험료','원수해약환급금','수재보험료','수재해약환급금','출재보험료','출재해약환급금','경과보험료','원수지급준비금','수재지급준비금','출재지급준비금','발생손해액','순사업비']
        filtered = rpt.merge(claf_auto, left_on=1, right_on='구분', how='inner').drop(2, axis=1)[cols]
        filtered.columns = colnames
        filtered = filtered.dropna()
        filtered['원수보험료'] = filtered['원수보험료'] - filtered['원수해약환급금']
        filtered['수재보험료'] = filtered['수재보험료'] - filtered['수재해약환급금']
        filtered['출재보험료'] = filtered['출재보험료'] - filtered['출재해약환급금']
        filtered.drop(['원수해약환급금','수재해약환급금','출재해약환급금'], axis=1, inplace=True)    
    return filtered

In [49]:
def rbc_sum(filtered):
    # 정제된 사업실적표를 RBC분류 단위로 합치는 함수
    summed = filtered.drop(['모니터링분류','구분'], axis=1).groupby('RBC분류').sum()
    return summed

In [53]:
def rbc_expo_prem(summed):
    # 누적 수입보험료
    return summed[['원수보험료','수재보험료','출재보험료']]
def rbc_expo_rsrv(summed):
    # 지급준비금
    return summed[['원수지급준비금','수재지급준비금','출재지급준비금']]
def rbc_comb(summed):
    # 누적 합산비율
    return summed[['경과보험료','발생손해액','순사업비']]

In [71]:
def rbc_expo_prem_1yr(num, date):
    # 직전 1년 수입보험료
    dates = genDates(date)
    if str(date)[2:] == '12':
        prem_expo = rbc_expo_prem(rbc_sum(fss_filter(num,dates[0])))
    else:
        prem_expo = rbc_expo_prem(rbc_sum(fss_filter(num,dates[0]))) + rbc_expo_prem(rbc_sum(fss_filter(num,dates[1]))) - rbc_expo_prem(rbc_sum(fss_filter(num,dates[2])))
    return prem_expo

def rbc_comb_3yrs(num, date):
    # 직전 3년 합산비율
    dates = genDates(date)
    if str(date)[2:] == '12':
        comb_0 = rbc_comb(rbc_sum(fss_filter(num,dates[0])))
        comb_1 = rbc_comb(rbc_sum(fss_filter(num,dates[1])))
        comb_2 = rbc_comb(rbc_sum(fss_filter(num,dates[2])))
    else:
        comb_0 = rbc_comb(rbc_sum(fss_filter(num,dates[0]))) + rbc_comb(rbc_sum(fss_filter(num,dates[1]))) - rbc_comb(rbc_sum(fss_filter(num,dates[2])))
        comb_1 = rbc_comb(rbc_sum(fss_filter(num,dates[2]))) + rbc_comb(rbc_sum(fss_filter(num,dates[3]))) - rbc_comb(rbc_sum(fss_filter(num,dates[4])))
        comb_2 = rbc_comb(rbc_sum(fss_filter(num,dates[4]))) + rbc_comb(rbc_sum(fss_filter(num,dates[5]))) - rbc_comb(rbc_sum(fss_filter(num,dates[6])))
            
    comb_0.columns = comb_0.columns + '_0년'
    comb_1.columns = comb_1.columns + '_1년'
    comb_2.columns = comb_2.columns + '_2년'
    comb = pd.concat([comb_0, comb_1, comb_2], axis=1)
    return comb

In [63]:
rbc_comb_3yrs(59,1803)

[AI059_1803]가 생성 되었습니다.
[AI059_1712]가 생성 되었습니다.
[AI059_1703]가 생성 되었습니다.
[AI059_1703]가 생성 되었습니다.
[AI059_1612]가 생성 되었습니다.
[AI059_1603]가 생성 되었습니다.
[AI059_1603]가 생성 되었습니다.
[AI059_1512]가 생성 되었습니다.
[AI059_1503]가 생성 되었습니다.


,경과보험료_0년,발생손해액_0년,순사업비_0년,경과보험료_1년,발생손해액_1년,순사업비_1년,경과보험료_2년,발생손해액_2년,순사업비_2년
RBC분류,,,,,,,,,
"1. 화재,기술,해외","202,229,131","135,865,437","83,510,914","201,932,491","150,585,752","78,328,797","193,885,338","128,836,606","68,963,551"
2. 종합,"69,938,232","29,731,192","22,202,407","64,606,188","44,540,813","16,114,045","57,411,098","31,518,357","13,689,601"
3. 해상,"16,428,540","7,108,518","5,533,448","19,650,352","12,693,627","5,299,084","15,609,929","21,558,956","7,823,097"
4. 상해,"114,833,352","90,439,170","18,467,977","96,368,476","80,194,499","21,283,636","92,494,953","86,941,580","19,077,676"
"5. 근재, 책임","66,461,771","37,191,543","11,781,118","69,282,015","49,426,195","12,675,690","63,562,691","42,793,647","15,089,904"
6. 기타일반,"78,870,538","55,637,067","11,250,517","73,801,427","53,910,179","8,985,956","58,553,825","36,655,960","7,573,882"


In [73]:
def rbc_result(num, date):
    # RBC 입력데이터
    return pd.concat([rbc_expo_prem_1yr(num, date), rbc_expo_rsrv(rbc_sum(fss_filter(num, date))), rbc_comb_3yrs(num, date)], axis=1)

In [77]:
rbc_result(60, 1803)

[AI060_1803]가 생성 되었습니다.
[AI060_1712]가 생성 되었습니다.
[AI060_1703]가 생성 되었습니다.
[AI060_1803]가 생성 되었습니다.
[AI060_1803]가 생성 되었습니다.
[AI060_1712]가 생성 되었습니다.
[AI060_1703]가 생성 되었습니다.
[AI060_1703]가 생성 되었습니다.
[AI060_1612]가 생성 되었습니다.
[AI060_1603]가 생성 되었습니다.
[AI060_1603]가 생성 되었습니다.
[AI060_1512]가 생성 되었습니다.
[AI060_1503]가 생성 되었습니다.


,원수보험료,수재보험료,출재보험료,원수지급준비금,수재지급준비금,출재지급준비금,경과보험료_0년,발생손해액_0년,순사업비_0년,경과보험료_1년,발생손해액_1년,순사업비_1년,경과보험료_2년,발생손해액_2년,순사업비_2년
RBC분류,,,,,,,,,,,,,,,
1. 개인용(대인),"1,727,149,263",0,"8,242,060","1,072,227,664",0,"4,297,053","1,535,640,276","1,690,264,084","285,905,884","1,535,640,276","1,690,264,084","285,905,884","1,535,640,276","1,690,264,084","285,905,884"
"2. 개인용(대물, 자차)","5,421,881,130",0,"29,513,079","362,580,150",0,"4,069,206","5,342,033,550","3,904,537,659","877,576,938","5,342,033,550","3,904,537,659","877,576,938","5,342,033,550","3,904,537,659","877,576,938"
3. 업무영업용(대인),"2,505,213,123",0,"9,852,616","1,697,431,589",0,"6,204,240","2,225,478,292","2,489,732,448","405,832,263","2,225,478,292","2,489,732,448","405,832,263","2,225,478,292","2,489,732,448","405,832,263"
"4. 업무영업용(대물, 자차)","9,036,468,550",0,"49,188,465","604,300,250",0,"6,782,010","8,903,389,250","6,507,562,765","1,462,628,230","8,903,389,250","6,507,562,765","1,462,628,230","8,903,389,250","6,507,562,765","1,462,628,230"
5. 기타,"2,052,630,405",0,"10,136,853","476,124,521",0,"3,232,875","1,958,048,759","1,368,701,743","564,065,318","1,958,048,759","1,368,701,743","564,065,318","1,958,048,759","1,368,701,743","564,065,318"
